# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc

We are creating analytics table which will be used by avialines to analyze avia trends.  imigration dataset and country codes 

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
We will be using next datasets:

- I94 Immigration Data: This data comes from the [US National Tourism and Trade Office.](https://www.trade.gov/national-travel-and-tourism-office) 
- Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).

In [3]:
# Read in the data here

#### Let's start spark session

In [1]:
from pyspark.sql import SparkSession

# config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:3.0.0-s_2.12").\
enableHiveSupport().getOrCreate()


21/10/24 14:16:43 WARN Utils: Your hostname, lwo1-lhp-a05809 resolves to a loopback address: 127.0.1.1; using 192.168.0.201 instead (on interface wlp0s20f3)
21/10/24 14:16:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://repos.spark-packages.org/ added as a remote repository with the name: repo-1


:: loading settings :: url = jar:file:/home/roman/Documents/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/roman/.ivy2/cache
The jars for the packages stored in: /home/roman/.ivy2/jars
saurfang#spark-sas7bdat added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-523d8d3b-119d-4308-9ef9-2be20fb3a4ed;1.0
	confs: [default]
	found saurfang#spark-sas7bdat;3.0.0-s_2.12 in spark-packages
	found com.epam#parso;2.0.11 in central
	found org.slf4j#slf4j-api;1.7.5 in local-m2-cache
	found org.apache.logging.log4j#log4j-api-scala_2.12;12.0 in central
	found org.scala-lang#scala-reflect;2.12.10 in user-list
	found org.apache.logging.log4j#log4j-api;2.13.2 in central
:: resolution report :: resolve 3282ms :: artifacts dl 17ms
	:: modules in use:
	com.epam#parso;2.0.11 from central in [default]
	org.apache.logging.log4j#log4j-api;2.13.2 from central in [default]
	org.apache.logging.log4j#log4j-api-scala_2.12;12.0 from central in [default]
	org.scala-lang#scala-reflect;2.12.10 from user-list in [default]
	org.slf4j#slf4j-api;1.7.5 from local-

#### Imigration data
TODO: fix link later

Excerpt from [data dictionary](/data/I94_SAS_Labels_Descriptions.SAS) 

##### Data dictionary
We won't go over all the details. Let's touch only details needed for analytics table

Column|Description| Key| 
------|----|
I94YR| 4 digit year |
I94MON| Numeric month 
I94CIT & I94RES| This format shows all the valid and invalid codes for processing | 
I94PORT| This format shows all the valid and invalid codes for processing works for you |
ARRDATE| Arrival Date in the USA. It is a SAS date numeric field that a permament format has not been applied. 
I94MODE| There are missing values as well as not reported (9) shows what we have found to be valid, everything else goes into 'other' works for you. 
I94ADDR| There is lots of invalid codes in this variable and the list below shows what we have found to be valid, everything else goes into 'other' | 
DEPDATE| Departure Date from the USA. It is a SAS date numeric field that a permament format has not been applied.  Please apply whichever date format works for you
I94BIR| Age of Respondent in Years 
I94VISA|Visa codes collapsed into three categories| 1 = Business, 2 = Pleasure, 3 = Student
COUNT| Used for summary statistics 
DTADFILE| Character Date Field - Date added to I-94 Files - CIC does not use 
VISAPOST| Department of State where where Visa was issued - CIC does not use 
OCCUP | Occupation that will be performed in U.S. - CIC does not use 
ENTDEPA | Arrival Flag - admitted or paroled into the U.S. - CIC does not use 
ENTDEPD | Departure Flag - Departed, lost I-94 or is deceased - CIC does not use 
ENTDEPU | Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use 
MATFLAG | Match flag - Match of arrival and departure records 
BIRYEAR | 4 digit year of birth 
DTADDTO | Character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use 
GENDER | Non-immigrant sex 
INSNUM | INS number 
AIRLINE | Airline used to arrive in U.S. 
ADMNUM | Admission Number 
FLTNO | Flight number of Airline used to arrive in U.S. 
VISATYPE | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. 

In [5]:
from os import listdir
from os.path import isfile, join
mypath="./data/18-83510-I94-Data-2016/"
sas_files = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
sas_files

['./data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat',
 './data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat']

#### Let's convert imigration data to spark friendly format
Note: as different files in the dataset contain different number of columns we will skip columns that are not in the data dictionary

In [4]:
import os
from pyspark.sql.types import *

for s in sas_files:
    df=spark.read.format('com.github.saurfang.sas.spark').load(s).write.mode("append").parquet("i94")
    

21/10/24 14:19:16 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


AnalysisException: The column number of the existing table default.i94(struct<cicid:double,i94yr:double,i94mon:double,i94cit:double,i94res:double,i94port:string,arrdate:double,i94mode:double,i94addr:string,depdate:double,i94bir:double,i94visa:double,count:double,dtadfile:string,visapost:string,occup:string,entdepa:string,entdepd:string,entdepu:string,matflag:string,biryear:double,dtaddto:string,gender:string,insnum:string,airline:string,admnum:double,fltno:string,visatype:string>) doesn't match the data schema(struct<cicid:double,i94yr:double,i94mon:double,i94cit:double,i94res:double,i94port:string,arrdate:double,i94mode:double,i94addr:string,depdate:double,i94bir:double,i94visa:double,count:double,validres:double,delete_days:double,delete_mexl:double,delete_dup:double,delete_visa:double,delete_recdup:double,dtadfile:string,visapost:string,occup:string,entdepa:string,entdepd:string,entdepu:string,matflag:string,biryear:double,dtaddto:string,gender:string,insnum:string,airline:string,ad

In [18]:
#write to parquet


# for s in sas_files:
spark.read.parquet("i94/").coalesce(1).write.parquet("avro_sas_data_final")


In [13]:
count=0
for s in sas_files:
    count+=spark.read.format('com.github.saurfang.sas.spark').load(s).count()
count

40790529

In [21]:
df = spark.read.parquet("avro_sas_data_final")

40790529

In [23]:
df.count()-count

0

In [25]:
len(df.schema)

28

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [5]:
df=df_spark.toPandas()
spark.stop()

21/10/23 21:35:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df_spark.count()

3096313

In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5749531.0,2016.0,4.0,251.0,251.0,NYC,20574.0,1.0,NV,20579.0,...,None,M,1974.0,10292016,F,None,DL,9.493189e+10,00469,B1
1,5749532.0,2016.0,4.0,251.0,251.0,NYC,20574.0,1.0,NV,20584.0,...,None,M,1976.0,10292016,F,None,DL,9.493207e+10,00469,B2
2,5749533.0,2016.0,4.0,251.0,251.0,NYC,20574.0,1.0,NV,20584.0,...,None,M,1977.0,10292016,M,None,DL,9.493198e+10,00469,B2
3,5749538.0,2016.0,4.0,251.0,251.0,NYC,20574.0,1.0,NY,NaN,...,None,None,1993.0,10292016,M,None,EK,9.495135e+10,00203,B2
4,5749540.0,2016.0,4.0,251.0,251.0,NYC,20574.0,1.0,NY,20575.0,...,None,M,1977.0,10292016,M,None,PR,9.502526e+10,00126,B2


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
 #   Column    Dtype  
---  ------    -----  
 0   cicid     float64
 1   i94yr     float64
 2   i94mon    float64
 3   i94cit    float64
 4   i94res    float64
 5   i94port   object 
 6   arrdate   float64
 7   i94mode   float64
 8   i94addr   object 
 9   depdate   float64
 10  i94bir    float64
 11  i94visa   float64
 12  count     float64
 13  dtadfile  object 
 14  visapost  object 
 15  occup     object 
 16  entdepa   object 
 17  entdepd   object 
 18  entdepu   object 
 19  matflag   object 
 20  biryear   float64
 21  dtaddto   object 
 22  gender    object 
 23  insnum    object 
 24  airline   object 
 25  admnum    float64
 26  fltno     object 
 27  visatype  object 
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [10]:
print(df.shape)

(3096313, 28)


In [11]:
df.describe(include='all')

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3096313,3.096313e+06,3.096074e+06,2943721,2.953856e+06,...,392,2957884,3.095511e+06,3095836,2682044,113708,3012686,3.096313e+06,3076764,3096313
unique,NaN,NaN,NaN,NaN,NaN,299,NaN,NaN,457,NaN,...,2,1,NaN,777,4,1913,534,NaN,7152,17
top,NaN,NaN,NaN,NaN,NaN,NYC,NaN,NaN,FL,NaN,...,U,M,NaN,07282016,M,3692,AA,NaN,LAND,WT
freq,NaN,NaN,NaN,NaN,NaN,485916,NaN,NaN,621701,NaN,...,391,2957884,NaN,67889,1377224,2155,310091,NaN,44297,1309059
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,NaN,2.055985e+04,1.073690e+00,NaN,2.057395e+04,...,NaN,NaN,1.974232e+03,NaN,NaN,NaN,NaN,7.082885e+10,NaN,NaN
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,NaN,8.777339e+00,5.158963e-01,NaN,2.935697e+01,...,NaN,NaN,1.742026e+01,NaN,NaN,NaN,NaN,2.215442e+10,NaN,NaN
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,NaN,2.054500e+04,1.000000e+00,NaN,1.517600e+04,...,NaN,NaN,1.902000e+03,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,NaN,2.055200e+04,1.000000e+00,NaN,2.056100e+04,...,NaN,NaN,1.962000e+03,NaN,NaN,NaN,NaN,5.603523e+10,NaN,NaN
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,NaN,2.056000e+04,1.000000e+00,NaN,2.057000e+04,...,NaN,NaN,1.975000e+03,NaN,NaN,NaN,NaN,5.936094e+10,NaN,NaN
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,NaN,2.056700e+04,1.000000e+00,NaN,2.057900e+04,...,NaN,NaN,1.986000e+03,NaN,NaN,NaN,NaN,9.350987e+10,NaN,NaN


In [12]:
def data_inv(df):
    print('number of records: ',df.shape[0])
    print('columns count: ',df.shape[1])
    print('-'*10)
    print('dateset columns: \n')
    print(df.columns)
    print('-'*10)
    print('data-type of each column: \n')
    print(df.dtypes)
    print('-'*10)
    print('missing rows in each column: \n')
    c=df.isnull().sum()
    print(c[c>0])
data_inv(df)

netflix movies and shows:  3096313
dataset variables:  28
----------
dateset columns: 

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')
----------
data-type of each column: 

cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port      object
arrdate     float64
i94mode     float64
i94addr      object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile     object
visapost     object
occup        object
entdepa      object
entdepd      object
entdepu      object
matflag      object
biryear     float64
dtaddto      object
gender       object
insnum       object
airline      object
admnum      float64
f

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [ ]:
spark.stop()